In [ ]:
!pip install imutils
!pip install keras-facenet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.9 MB/s eta 0:00:00
  Created wheel for keras-facenet: filename=keras_facenet-0.3.2-py3-none-any.whl size=10385 sha256=bb7e7f0cd32c300e56664fb909cc96236444f07681ad526ea3786ee40b68edcb
  Stored in directory: /root/.cache/pip/wheels/73/5d/41/90b5d28ca667cfc4748ae859fa4f0b85b936d73207a073ded5
Successfully built keras-facenet


In [ ]:
import cv2
import numpy as np
from imutils.video import FileVideoStream

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!wget https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt
!wget https://github.com/opencv/opencv_3rdparty/raw/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel

--2023-04-19 17:53:21--  https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28104 (27K) [text/plain]
Saving to: ‘deploy.prototxt’

deploy.prototxt     100%[===================>]  27.45K  --.-KB/s    in 0.002s  

2023-04-19 17:53:21 (12.0 MB/s) - ‘deploy.prototxt’ saved [28104/28104]

--2023-04-19 17:53:21--  https://github.com/opencv/opencv_3rdparty/raw/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/opencv/opencv_3rdparty/dnn_s

In [ ]:
INFILE = "/content/drive/MyDrive/FR_Colab/hh.mp4"
OUTFILE = "/content/drive/MyDrive/FR_Colab/hh_processed.mp4"
DATASET_DIR = "/content/drive/MyDrive/FR_Colab/face_dataset"
MODEL = "/content/drive/MyDrive/FR_Colab/built_model.h5"

In [ ]:
import datetime
from IPython.display import clear_output

# Load the face detection model
model_file = 'deploy.prototxt'
model_weights = 'res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(model_file, model_weights)

# Start the video stream
vs = FileVideoStream(INFILE).start()

# Get the total number of frames in the video
total_frames = int(vs.stream.get(cv2.CAP_PROP_FRAME_COUNT))

# Get the video writer ready
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = vs.stream.get(cv2.CAP_PROP_FPS)
width = int(vs.stream.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vs.stream.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(OUTFILE, fourcc, fps, (width, height))# --OUTPUT VIDEO PATH!!!

# Loop through the frames in the video stream
while True:
    # Read the next frame from the video stream
    frame = vs.read()

    # If there are no more frames, break out of the loop
    if frame is None:
        break

    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 0.5, (h,h), (104.0, 177.0, 123.0))

    net.setInput(blob)
    detections = net.forward()

    for i in range(0, detections.shape[2]):

        confidence = detections[0, 0, i, 2]

        if confidence < 0.6:
            continue

        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")


        #UNCOMMENT THIS PART FOR DATASET COLLECTION----------------------------------------------------------
        #img = frame[startY:endY,startX:endX]
        #try:
        #    cv2.imwrite("/content/drive/MyDrive/face_dataset/"+str(datetime.datetime.now())+".jpg",img)
        #except:
        #    print("imwrite error")


        cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 0, 255), 2)


    # Write the frame with bounding boxes to the output video file
    out.write(frame)

    clear_output(wait=True)
    frame_num = int(vs.stream.get(cv2.CAP_PROP_POS_FRAMES))
    # Calculate the progress percentage
    progress_pct = int((frame_num / total_frames) * 100)
    # Print the progress in the console
    print(f'Frame {frame_num} /{total_frames} ({progress_pct}% complete)')

    # Wait for a key event
    key = cv2.waitKey(1) & 0xFF

    # If the 'q' key is pressed, break out of the loop
    if key == ord('q'):
        break

# Release the video stream and video writer
vs.stop()
out.release()

# Destroy all windows
cv2.destroyAllWindows()

Frame 0 /5856 (0% complete)


In [ ]:
#RUN FOR FACE RECOGNITION PREP-----------------------------------------------------------------------------------------
import tensorflow as tf
from keras_facenet import FaceNet
from keras.models import load_model
import csv
import os
import datetime
from IPython.display import clear_output

# Set the directory path
directory_path = DATASET_DIR

# Get the list of all folders in the directory
label = [folder for folder in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, folder))]
print(label)

facenet_model = FaceNet()
fmodel = load_model(MODEL)

def recFace(image):

    #lbl = ""
    max = -1

    if np.any(image):

        try:

            global facenet_model, fmodel

            image = cv2.resize(image,(224,224))
            image = tf.keras.preprocessing.image.img_to_array(image)

            image = tf.constant(image)
            image = tf.keras.backend.cast(image, 'float32')
            image = tf.keras.backend.reshape(image, (1, 224, 224, 3))
            image = tf.keras.backend.concatenate([image, image], axis=0)

            imgs = [image]

            embeddings = facenet_model.embeddings(imgs)

            embeddings = tf.constant(embeddings)
            embeddings = tf.keras.backend.cast(embeddings, 'float32')

            pred = fmodel.predict(embeddings)
            max = np.argmax(pred[0])

            print(pred)

            if pred[0][max] < 0.6:
                #lbl = "Unknown"
                max = -1

        except:
            print('frame error')

    return max#, lbl

['bodyguard', 'man1', 'man2', 'masked guard', 'old lady', 'man3', 'roshan', 'blackbeard man', 'target', 'basim', 'boy', 'greybeard man', 'man5', 'helmet guard', 'fakir', 'dark lady', 'unfortunate vendor', 'man4', 'wmn1', 'beard-only man', 'dark man', 'young lady', 'unmasked guard']


In [ ]:
#RUN FOR FACE RECOGNITION-----------------------------------------------------------------------------------------

# Load the face detection model
model_file = 'deploy.prototxt'
model_weights = 'res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(model_file, model_weights)

# Start the video stream
vs = FileVideoStream(INFILE).start()

# Get the total number of frames in the video
total_frames = int(vs.stream.get(cv2.CAP_PROP_FRAME_COUNT))

# Get the video writer ready
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = vs.stream.get(cv2.CAP_PROP_FPS)
width = int(vs.stream.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vs.stream.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(OUTFILE, fourcc, fps, (width, height))

# Loop through the frames in the video stream
while True:
    # Read the next frame from the video stream
    frame = vs.read()

    # If there are no more frames, break out of the loop
    if frame is None:
        break

    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 0.5, (h,h), (104.0, 177.0, 123.0))

    net.setInput(blob)
    detections = net.forward()

    for i in range(0, detections.shape[2]):

        confidence = detections[0, 0, i, 2]

        if confidence < 0.6:
            continue

        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")

        
        img = frame[startY:endY,startX:endX]
        face_id = recFace(img)
        if face_id != -1:
            face = label[face_id][0]
        else:
            face = "Unknown"
        cv2.putText(frame,face,(startX-10,startY-5),cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,0,255),2,cv2.LINE_AA)


        cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 0, 255), 2)

    # Write the frame with bounding boxes to the output video file
    out.write(frame)

    clear_output(wait=True)
    frame_num = int(vs.stream.get(cv2.CAP_PROP_POS_FRAMES))
    # Calculate the progress percentage
    progress_pct = int((frame_num / total_frames) * 100)
    # Print the progress in the console
    print(f'Frame {frame_num} /{total_frames} ({progress_pct}% complete)')

    # Wait for a key event
    key = cv2.waitKey(1) & 0xFF

    # If the 'q' key is pressed, break out of the loop
    if key == ord('q'):
        break

# Release the video stream and video writer
vs.stop()
out.release()

# Destroy all windows
cv2.destroyAllWindows()